In [ ]:
#Packages
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pymks import (generate_multiphase, plot_microstructures, PrimitiveTransformer,
                   TwoPointCorrelation,FlattenTransformer, PrimitiveBasis)
from pymks.stats import correlate
from pymks.tools import draw_microstructures
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob
import pandas as pd
import csv

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
#Append images all to one list.
images = []
for filename in glob.glob(r'C:\Users\mikep\Desktop\project\data\micress\*.tif'):
    im=Image.open(filename)
    images.append(im)
np.shape(images[0])

In [ ]:
#List comprehension to convert the members of the "images" list to arrays.
imarrays = [np.array(x) for x in images]
imarrays[0].shape

In [ ]:
#Stacking all image arrays to create the 'sample' dimension and to convert the list to an array.
data_a = np.stack(images, axis=0)
data_a.shape

In [ ]:
#Visualization of the microstructures.
plot_microstructures(*data_a[:10], cmap='gray', colorbar=False);

In [ ]:
#List comprehension to add the "phases" axis with the PrimitiveTransformer class.
data = PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(data_a)
data.shape

In [ ]:
data_corr = TwoPointCorrelation(
    periodic_boundary=True,
    cutoff=10,
    correlations=[(0, 0), (0, 1)]
).transform(data)

data_corr.shape

#for i in range(0, 11):
    #print(data_corr[i, :, :, 0])

In [ ]:
plot_microstructures(
    data_corr[0, :, :, 0],
    data_corr[0, :, :, 1],
    titles=['Auto-correlation', 'Cross-correlation'],
    showticks=True
);

In [ ]:
pca = PCA(
    svd_solver='full',
    n_components=2,
    random_state=10
)

pc_scores=pca.fit_transform(data_corr.reshape(10, -1))
pc_scores.shape

In [ ]:
#PYTEST_VALIDATE_IGNORE_OUTPUT
%matplotlib notebook

In [ ]:
pca.components_



In [ ]:
for x in range(len(pc_scores)):
    print(pc_scores[x])

In [ ]:
#PYTEST_VALIDATE_IGNORE_OUTPUT
pc1, pc2 = np.split(pc_scores, 2)
print(pc1, pc2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
#, projection='2d')

ax.scatter(pc1[:,0], pc1[:,1], c='red', marker='o')
ax.scatter(pc2[:,0], pc2[:,1], c='blue', marker='o')
#ax.scatter(pc3[:,0], pc3[:,1], pc3[:,2], c='b', marker='o')

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
#ax.set_zlabel("PC3")

plt.grid(color = 'black', linestyle = '--', linewidth = 0.5)
plt.legend()

plt.show()

In [ ]:
# write csv file
f = open(r'C:\Users\mikep\Desktop\project\data\pc-scores\pc1.csv', 'w')
writer = csv.writer(f)
writer.writerow(pc1)
f.close()